# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities=pd.read_csv("../WeatherPy/cities.csv")
cities

,City,Max Temp,Cloudiness,Latitude,Longitude,Humidity,Wind Speed,Country,Date
0,punta arenas,69.80,0,-53.1500,-70.9167,52,5.75,CL,1612904192
1,vila velha,78.01,90,-20.3297,-40.2925,94,5.75,BR,1612904363
2,bredasdorp,64.40,11,-34.5322,20.0403,82,2.30,ZA,1612904363
3,meulaboh,77.59,15,4.1363,96.1285,85,4.59,ID,1612904363
4,vaini,80.60,90,-21.2000,-175.2000,89,21.85,TO,1612904364
...,...,...,...,...,...,...,...,...,...
545,puerto madryn,75.31,100,-42.7692,-65.0385,34,23.76,AR,1612904516
546,mednogorsk,-19.28,62,51.4128,57.5950,87,3.53,RU,1612904517
547,bouloupari,77.00,75,-21.8667,166.0500,88,2.30,NC,1612904517
548,san policarpo,78.73,100,12.1791,125.5072,86,12.97,PH,1612904517


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=cities[['Latitude','Longitude']]
hum=cities['Humidity'].astype(float)

In [5]:
fig = gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=hum)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_df=cities.loc[(cities['Max Temp'] <80) & (cities['Max Temp']>70)]
new_df=new_df.loc[new_df['Wind Speed']<10]
ideal_weather_df=new_df.loc[new_df['Cloudiness']==0]
ideal_weather_df

,City,Max Temp,Cloudiness,Latitude,Longitude,Humidity,Wind Speed,Country,Date
38,east london,72.03,0,-33.0153,27.9116,77,3.96,ZA,1612904371
72,salalah,75.20,0,17.0151,54.0924,60,1.14,OM,1612904380
75,olavarria,75.00,0,-36.8927,-60.3225,45,5.99,AR,1612904380
154,chuy,72.46,0,-33.6971,-53.4616,75,9.10,UY,1612904394
160,keita,77.00,0,14.7551,5.7749,12,4.61,NE,1612904395
351,bilma,70.57,0,18.6853,12.9164,19,9.13,NE,1612904478
415,broken hill,71.60,0,-31.9500,141.4333,28,6.91,AU,1612904491


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
base_url='http://maps.googleapis.com.maps/api/place/nearbysearch/json'
hotels=[]
for index,row in ideal_weather_df.iterrows():
    target_coordinates=f"{row['Latitude']}, {row['Longitude']}"
    target_search='Hotel'
    target_radius=5000
    target_type='lodging'
    params={'location':target_coordinates,
       'keyword':target_search,
       'radius':target_radius,
       'type':target_type,
       'key':g_key}
    #try:
    response=requests.get(base_url, params=params,headers={'content-type':'application/json'})
    response.request.url
    places_data=response.json()
    print(places_data['results'][0])
    hotels.append(places_data['results'][0]['name'])
    #except:
        #hotels.append('')
        
ideal_weather_df['Hotel']=hotels


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [12]:
print(response.text)

In [ ]:
response.json()

In [ ]:
places_data=response.json()

print(json.dumps(places_data, indent=4,sort_keys=True))

In [ ]:
places_data['results'][0].keys()

In [ ]:
places_data['results'][0]['name']

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
